In [34]:
import data 

shpFile = data.getGeoData("cebu")

File cebu_geometry.shp already downloaded.


In [36]:
# load shapefile data
import shapefile

shapeData = shapefile.Reader(shpFile)
records = shapeData.shapeRecords()

# print number of records 
print "Total shape records: " + str(len(records))


Total shape records: 47675


In [39]:
# load bounding box from geojson file, created using geojson.io 
# https://gist.github.com/anonymous/3d160c1955a8d31fd5f8d5e298248cde

import json
from shapely.geometry import mapping, shape

# load geojson file from project
with open ("example2.geojson", "r") as myfile:
    featureCollection=myfile.read()
    
    # geojson.io creates FeatureCollection, extracting the first geometry object from the collection
    geoJsonData = json.loads(featureCollection)['features'][0]['geometry']
    
    # createa shape 
    boundingBox = shape(geoJsonData)
  
print boundingBox

POLYGON ((123.888144493103 10.30563036038582, 123.9008259773254 10.30563036038582, 123.9008259773254 10.31789590621363, 123.888144493103 10.31789590621363, 123.888144493103 10.30563036038582))


In [17]:
# optional: visualize geoJson bounding box (requries ipyleaflet module)

from ipyleaflet import (
    Map,
    GeoJSON
)

center = [boundingBox.centroid.y, boundingBox.centroid.x]

# create 
mapObj = Map(center=center, zoom=13)

geoJsonLayer = GeoJSON(data=geoJsonData)
mapObj.add_layer(geoJsonLayer)

#display map
mapObj

In [42]:
from shapely.geometry import mapping, shape

edgeIds = dict()

for shapeRecord in records:
    s = shape(shapeRecord.shape)
    if boundingBox.intersects(s):
        edgeIds[shapeRecord.record[0]] = True
        
print "Shapes within bounding box: " + str(len(edgeIds))
        

Shapes within bounding box: 736


In [49]:
# calculate average speed by day of week for edges found within bounding box

import numpy, csv

# load csv file and calculate daily average speed

days = ['Mon', 'Tues', 'Weds', 'Thurs', 'Fri', 'Sat', 'Sun']

# download weekly data file
path = data.getData("cebu", "2015-12-28")

with open(path, "rb") as csvfile:
    dataReader = csv.reader(csvfile, delimiter=',')
    # return to top of file
    csvfile.seek(0)

    # skip csv header
    next(dataReader, None)

    # create daily average speed
    # In CSV day_of_week is a number from 0 to 6 representing day of week (0 = monday, 6=sunday)

    # there are lots of rows in the files
    # not feasible to keep all the numbers in memroy, let's calculate a running average instead
    dayOfWeekCount = numpy.zeros(7)
    dayOfWeekSum = numpy.zeros(7)

    dayOfWeekAvg = numpy.zeros(7)

    print "Processing..."

    rowCount = 0
    for row in dataReader:
        
        # only count rows with edgeIds found within the bounding box
        if int(row[1]) in edgeIds:
            # increment the count for day of week
            dayOfWeekCount[int(row[2])] += 1

            #increment sum for speed
            dayOfWeekSum[int(row[2])] += float(row[4])

    csvfile.close()

    print "Average speeds for matching edges by day (kph):"

    # loop over arrays and calculate averages
    for day in range(0,7):
        if dayOfWeekCount[day] == 0:
            print days[day] + ": (no data)"
        else:
            averageSpeed = dayOfWeekSum[day] / dayOfWeekCount[day]

            dayOfWeekAvg[day] = averageSpeed
            print days[day] + ": " + str(averageSpeed)

File data/cebu/week_2015-12-28.csv already downloaded.
Processing...
Average speeds for matching edges by day (kph):
Mon: 20.4482250608
Tues: 20.1066177024
Weds: 20.0799784493
Thurs: 20.1287169318
Fri: 20.4148617169
Sat: 21.1637792642
Sun: 22.4987028177
